In [14]:
import pandas as pd 

# Reading all data files as the CSV

books_df = pd.read_csv('books_dataset.csv')
members_df = pd.read_csv('members_dataset.csv')
transaction_df = pd.read_csv('transactions_dataset.csv')

# **Basic Analysis**  : Books Dataframe #   

In [ ]:

# Checking books_df shape (Rows and columns )
books_df.shape

# Checking info for the books_df 
books_df.info()

# Checking for null values 
print(f"Null values : {books_df.isnull().sum()}")

# Checking for duplication 
print(f"Duplicate values : {books_df.duplicated().sum()}")

# Checking for duplication 
print(f"Statistics Summary : {books_df.describe()}")

# No. of authors in dataset 
# unique() when you need to see the list of distinct values themselves
# Use nunique() when you need to know the number of distinct values.
print(f"No of authors in the dataset : {books_df["Author"].nunique()}")

# No. of Genre in the dataset 
print(f"No of Genre in the dataset : {books_df["Genre"].nunique()}")

# Book_count by Genre : Frequency 
# value_counts() : Return a Series containing counts of unique values.
print(f"Genre distribution:{books_df["Genre"].value_counts()}")

# Book_count by Author : frequecy 
print(f"Author distribution: {books_df["Author"].value_counts()}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Book_ID           500 non-null    object
 1   Title             500 non-null    object
 2   Author            500 non-null    object
 3   Genre             500 non-null    object
 4   Publication_Year  500 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 19.7+ KB
Null values : Book_ID             0
Title               0
Author              0
Genre               0
Publication_Year    0
dtype: int64
Duplicate values : 0
Statistics Summary :        Publication_Year
count        500.000000
mean        2021.264000
std            1.827836
min         2012.000000
25%         2020.000000
50%         2022.000000
75%         2023.000000
max         2024.000000
No of authors in the dataset : 34
No of Genre in the dataset : 25
Genre distribution:Genre
Sociology     29
Health        28
Edu

### 1. **Most Popular Books and Authors**

In [ ]:

# 1. Which books have been borrowed the most times?

Books_count = transaction_df["Book_ID"].value_counts()

# print(Books_count)
# left_index = true This tells Pandas to use the index of the left object (Books_count) as the key for merging.
# right_on="Book_ID" : This tells Pandas to use the "Book_ID" column of books_df as the key for merging.

Most_popular_books = pd.merge(Books_count.rename("Most popular Books"),books_df,left_index=True,right_on="Book_ID")
print(Most_popular_books)


# 2. Who are the top 10 most popular authors?

# Popular authors by the readers (by total borrows)
unique_authors = set()

for name in Most_popular_books["Author"]:
    unique_authors.add(name)

print(unique_authors)

# Popular authors by the No. of books written 

Author_Book_count = books_df["Author"].value_counts().head(10) 
print(Author_Book_count)


# 3. Are newer books more popular than older ones?
# By Count classifying newer book by publication year > 2020

book_popularity = books_df.merge(
    Books_count.rename("Borrow_Count"),
    on="Book_ID",
    how="left"
)

# if Borrow count has Nan then filled with 0 
book_popularity["Borrow_Count"] = book_popularity["Borrow_Count"].fillna(0)

# Comparing based on the puvblication year newer and older books 
new_books_popularity = book_popularity[book_popularity["Publication_Year"] > 2020]["Borrow_Count"].sum()
old_books_popularity = book_popularity[book_popularity["Publication_Year"] <= 2020]["Borrow_Count"].sum()

if new_books_popularity > old_books_popularity:
    print("Newer books are more popular than older books.")
else:
    print("Older books are more popular than newer books.")


# 4.Do certain genres get borrowed more frequently?

genre_popularity = (
    book_popularity.groupby("Genre")["Borrow_Count"]
    .sum()
    .sort_values(ascending=False)
)

print(genre_popularity.head(10))

     Most popular Books Book_ID                  Title            Author  \
11                   48    B012       The Desert Story      James Miller   
191                  45    B192    Beauty: A Biography  Dr. Amara Okafor   
271                  41    B272    Desert: A Biography        Rachel Kim   
143                  40    B144            Tech Beauty      Kai Nakamura   
194                  39    B195          The War Story    Oliver Fischer   
..                  ...     ...                    ...               ...   
359                   1    B360    The Wisdom Analysis       Lucas Silva   
129                   1    B130  The Creation Analysis     Sophie Martin   
192                   1    B193    The Knowledge Study         Lisa Wang   
402                   1    B403       The Ice Handbook        Rachel Kim   
412                   1    B413         The Ice Manual       Yuki Tanaka   

          Genre  Publication_Year  
11    Biography              2024  
191   Biography

# **Basic Analysis** : Member Dataframe #

In [17]:
# Checking shape of a member 
print("Shape:", members_df.shape)

# Checking member info 
print(members_df.info())

# Checking for th emissing values 
print("Missing values:\n", members_df.isnull().sum())

# Checking for the duplications 
print("Duplicate rows:", members_df.duplicated().sum())

#Checking Membership type 
print("Unique Membership Types:", members_df["Membership_Type"].nunique())

#Checking forAge values 
print("Unique Age Values:", members_df["Age"].nunique())

# Statistical distriution of member 
print(members_df.describe())

# Membership type and there count 
print(members_df["Membership_Type"].value_counts())

# Classifying by the age groups 
bins = [15, 19, 29, 49, 64, 120]
labels = ["Teens", "Young Adults", "Adults", "Middle-aged", "Seniors"]

members_df["Age_Group"] = pd.cut(members_df["Age"], bins=bins, labels=labels)
print(members_df["Age_Group"].value_counts())

# Member Segmentation 
# Borrow Count per member 
borrow_count = transaction_df["Member_ID"].value_counts().rename("Borrow_Count")

# Merge Member info with borrow count 
member_activity = members_df.merge(borrow_count, on="Member_ID", how="left")

# Replace NaN with 0 for non-borrowers
member_activity["Borrow_Count"] = member_activity["Borrow_Count"].fillna(0).astype(int)

print(member_activity)

# used to convert numeric values into catagorical 
member_activity["User_Type"] = pd.cut(
    member_activity["Borrow_Count"],
    bins=[-1, 4, 9, 20, float("inf")],
    labels=["Inactive", "Light", "Regular", "Heavy"]
)

# printing first few rows 
print(member_activity.head())

# Summary count of each user type 
print(member_activity["User_Type"].value_counts())



Shape: (500, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_ID        500 non-null    object
 1   Name             500 non-null    object
 2   Age              500 non-null    int64 
 3   Membership_Type  500 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB
None
Missing values:
 Member_ID          0
Name               0
Age                0
Membership_Type    0
dtype: int64
Duplicate rows: 0
Unique Membership Types: 5
Unique Age Values: 63
              Age
count  500.000000
mean    39.174000
std     17.408902
min     16.000000
25%     24.000000
50%     36.000000
75%     54.250000
max     84.000000
Membership_Type
Regular    198
Student    148
Premium    105
Faculty     25
Senior      24
Name: count, dtype: int64
Age_Group
Young Adults    144
Adults          141
Middle-aged     110
Teens            53
Seni

# **Basic Analysis** : Transaction Dataframe #

In [18]:

# Transaction overview : 
print("Transaction DataFrame Overview:")
print(transaction_df.head())

# Transaction Information 
print("\nInfo:")
print(transaction_df.info())

# statistics summary of the Transaction dataset
print("\nSummary Statistics:")
print(transaction_df.describe())

# Missing values in the transaction dataset 
print("\nMissing Values:")
print(transaction_df.isna().sum())

# Total borrowings of the transaction dataset 
total_borrowings = len(transaction_df)
print("\nTotal Borrowings:", total_borrowings)

unique_members = transaction_df["Member_ID"].nunique()
unique_books = transaction_df["Book_ID"].nunique()

# Unique Members 
print("Unique Members:", unique_members)

# Unique Books 
print("Unique Books:", unique_books)

Transaction DataFrame Overview:
  Book_ID Member_ID Checkout_Date Return_Date
0    B082      M259    2023-01-01         NaN
1    B322      M411    2023-01-01  2023-01-10
2    B388      M025    2023-01-01  2023-01-22
3    B253      M005    2023-01-01         NaN
4    B056      M358    2023-01-01         NaN

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6687 entries, 0 to 6686
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Book_ID        6687 non-null   object
 1   Member_ID      6687 non-null   object
 2   Checkout_Date  6687 non-null   object
 3   Return_Date    5741 non-null   object
dtypes: object(4)
memory usage: 209.1+ KB
None

Summary Statistics:
       Book_ID Member_ID Checkout_Date Return_Date
count     6687      6687          6687        5741
unique     495       500           676         672
top       B012      M001    2023-05-16  2024-11-06
freq        48        17            19         12

### 2. **Member Usage Patterns** 

In [19]:

# 1. Which membership types borrow the most books?

membership_popularity = (
    member_activity.groupby("Membership_Type")["Borrow_Count"]
    .sum()
    .sort_values(ascending=False)
)

print("\nMembership Popularity:\n", membership_popularity)


# How does borrowing frequency vary by age group?

age_group_borrowing = (
    member_activity.groupby("Age_Group")["Borrow_Count"]
    .sum()
    .sort_values(ascending=False)
)

print("\nBorrowing by Age Group:\n", age_group_borrowing)

# Are there seasonal borrowing patterns?

# Convert to the data time 
transaction_df["Borrow_Date"] = pd.to_datetime(transaction_df["Checkout_Date"])

# Extract Month and Season 
transaction_df["Month"] = transaction_df["Borrow_Date"].dt.month  # .dt.month gives the month as an integer (1–12)
transaction_df["Season"] = transaction_df["Borrow_Date"].dt.quarter # .dt.quarter gives the quarter (1–4)

# Monthly Borrowing 
monthly_borrowing = (
    transaction_df["Month"]
    .value_counts()
    .sort_index()
)

print("Monthly Borrowing:\n", monthly_borrowing)


# Seasonal Borrowing 
seasonal_borrowing = (
    transaction_df["Season"]
    .value_counts()
    .sort_index()
)

print("\nSeasonal Borrowing (Quarterly):\n", seasonal_borrowing)


# Who are the most active vs inactive members?

# Most Active users
top_active_users = (
    member_activity.sort_values(by="Borrow_Count", ascending=False)
    .head(10)
)

print("\nTop 10 Most Active Users:\n", top_active_users)

# Most Inactive Users
inactive_users = (
    member_activity.sort_values(by="Borrow_Count", ascending=True)
    .head(10)
)

print("\nTop 10 Most Inactive Users:\n",inactive_users)

# Count of active inactive members 
summary = member_activity["User_Type"].value_counts()
print("\n User Type Distribution:")
print(summary)


member_activity["User_Type"].value_counts(normalize=True)



Membership Popularity:
 Membership_Type
Regular    2412
Student    2197
Premium    1580
Faculty     324
Senior      174
Name: Borrow_Count, dtype: int64

Borrowing by Age Group:
 Age_Group
Young Adults    2051
Adults          1856
Middle-aged     1393
Teens            792
Seniors          595
Name: Borrow_Count, dtype: int64
Monthly Borrowing:
 Month
1     598
2     556
3     630
4     611
5     644
6     590
7     594
8     630
9     600
10    590
11    370
12    274
Name: count, dtype: int64

Seasonal Borrowing (Quarterly):
 Season
1    1784
2    1845
3    1824
4    1234
Name: count, dtype: int64

Top 10 Most Active Users:
    Member_ID              Name  Age Membership_Type     Age_Group  \
2       M003        Aisha Khan   28         Premium  Young Adults   
4       M005        Sara Mehta   19         Student         Teens   
1       M002        Ravi Patel   34         Regular        Adults   
0       M001       Neha Sharma   22         Student  Young Adults   
5       M006        

C:\Users\Sakshi Ladkat\AppData\Local\Temp\ipykernel_16132\1060674056.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  member_activity.groupby("Age_Group")["Borrow_Count"]


User_Type
Regular     0.886
Light       0.102
Inactive    0.012
Heavy       0.000
Name: proportion, dtype: float64

### 3. **Overdue Analysis** 

In [20]:
# Current overdue books

# Today's date 
today = pd.Timestamp.today()

# Convert to date 
transaction_df["Checkout_Date"] = pd.to_datetime(transaction_df["Checkout_Date"])
transaction_df["Return_Date"] = pd.to_datetime(transaction_df["Return_Date"])

# Calculate days borrowed
transaction_df["Days_Borrowed"] = (
    transaction_df["Return_Date"].fillna(today) - transaction_df["Checkout_Date"]
).dt.days

# Books overdue if NOT returned OR returned after 30 days
overdue_books = transaction_df[
    (transaction_df["Days_Borrowed"] > 30)
]

print("\nOverdue Books (>30 days):")
print(overdue_books)

# Un-returned Books 
unreturned_books = transaction_df[transaction_df["Return_Date"].isna()]

print("\nUnreturned Books:")
print(unreturned_books)


# returned Patterns 
returned_only = transaction_df[transaction_df["Return_Date"].notna()]

avg_loan_period = returned_only["Days_Borrowed"].mean()

print("\nAverage Loan Period (in days):", round(avg_loan_period, 2))

# MEMBER RELIABILITY
# Tag late returns (returned after 30 days)
returned_only["Late_Return"] = returned_only["Days_Borrowed"] > 30

# Merge with members table
member_usage = returned_only.merge(
    members_df[["Member_ID", "Membership_Type"]],
    on="Member_ID",
    how="left"
)

# Reliability = % of books returned on time

# Realiability check by values.count() function normalized attribute 
grouped = member_usage.groupby("Membership_Type")["Late_Return"]
counts = grouped.value_counts(normalize=True).sort_values(ascending=False)
print(counts)


# Reliability check by the lambada function 
reliability = (
    member_usage.groupby("Membership_Type")["Late_Return"]
    .apply(lambda x: 100 - (x.mean() * 100))  # on-time percentage
    .sort_values(ascending=False)
    .rename("On_Time_Return_%")
)

print("\nMember Reliability (On-time return %):")
print(reliability)



Overdue Books (>30 days):
     Book_ID Member_ID Checkout_Date Return_Date Borrow_Date  Month  Season  \
0       B082      M259    2023-01-01         NaT  2023-01-01      1       1   
3       B253      M005    2023-01-01         NaT  2023-01-01      1       1   
4       B056      M358    2023-01-01         NaT  2023-01-01      1       1   
5       B361      M457    2023-01-01  2023-02-01  2023-01-01      1       1   
18      B121      M224    2023-01-02         NaT  2023-01-02      1       1   
...      ...       ...           ...         ...         ...    ...     ...   
6670    B300      M124    2024-11-05         NaT  2024-11-05     11       4   
6673    B486      M264    2024-11-05         NaT  2024-11-05     11       4   
6677    B470      M373    2024-11-06         NaT  2024-11-06     11       4   
6681    B390      M085    2024-11-06         NaT  2024-11-06     11       4   
6683    B329      M119    2024-11-06         NaT  2024-11-06     11       4   

      Days_Borrowed  
0 

C:\Users\Sakshi Ladkat\AppData\Local\Temp\ipykernel_16132\2054304688.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  returned_only["Late_Return"] = returned_only["Days_Borrowed"] > 30


4. **Genre Preferences by Age Group** 

In [23]:
# Expected Patterns
expected_patterns = {
    "Teens (16-19)": ["Education", "Science", "Technology", "Fiction", "Adventure"],
    "Young Adults (20-29)": ["Business", "Technology", "Biography", "Fiction", "Self-Help"],
    "Adults (30-49)": ["History", "Biography", "Business", "Health", "Philosophy"],
    "Middle-aged (50-64)": ["History", "Biography", "Philosophy", "Health", "Travel"],
    "Seniors (65+)": ["History", "Biography", "Philosophy", "Religion", "Art"]
}


# Merge Member, Transaction, Book info
merged_df = (
    transaction_df
    .merge(members_df[["Member_ID", "Age_Group"]], on="Member_ID", how="left")
    .merge(books_df[["Book_ID", "Genre"]], on="Book_ID", how="left")
)

# Count books borrowed by each age group per genre
age_genre_counts = (
    merged_df.groupby(["Age_Group", "Genre"])["Book_ID"]
    .count()
    .reset_index(name="Borrow_Count")
)

print(age_genre_counts)


# Identify Top Genres Borrowed by Each Age Group
top_genres_by_age = (
    age_genre_counts
    .sort_values(["Age_Group", "Borrow_Count"], ascending=[True, False])
    .groupby("Age_Group")
    .head(5)
)

print(top_genres_by_age)



    Age_Group       Genre  Borrow_Count
0       Teens   Adventure           122
1       Teens         Art             5
2       Teens   Biography            15
3       Teens    Business             6
4       Teens     Cooking            11
..        ...         ...           ...
120   Seniors   Sociology             9
121   Seniors      Sports             4
122   Seniors  Technology            11
123   Seniors    Thriller             4
124   Seniors      Travel            63

[125 rows x 3 columns]
        Age_Group       Genre  Borrow_Count
22          Teens  Technology           139
6           Teens   Education           132
0           Teens   Adventure           122
18          Teens     Science           117
8           Teens     Fiction            81
31   Young Adults   Education           348
47   Young Adults  Technology           348
33   Young Adults     Fiction           271
25   Young Adults   Adventure           262
43   Young Adults     Science           217
52         A

C:\Users\Sakshi Ladkat\AppData\Local\Temp\ipykernel_16132\2205318486.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  merged_df.groupby(["Age_Group", "Genre"])["Book_ID"]
C:\Users\Sakshi Ladkat\AppData\Local\Temp\ipykernel_16132\2205318486.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("Age_Group")


5. **Library Utilization Optimization** 

In [22]:
# Which books should be purchased in multiple copies?

# Count borrow frequency for each book
book_demand = (
    transaction_df.groupby("Book_ID")["Checkout_Date"]
    .count()
    .reset_index(name="Times_Borrowed")
)

# Merge with book information
book_demand = book_demand.merge(
    books_df[["Book_ID", "Title", "Genre"]],
    on="Book_ID", how="left"
)

# Books recommended for multiple copies
high_demand_books = book_demand.sort_values(
    by="Times_Borrowed", ascending=False
).head(20)

print(high_demand_books)

# What genres are underrepresented in borrowing?

# Count borrow frequency by genre
genre_borrow_counts = (
    merged_df.groupby("Genre")["Book_ID"]
    .count()
    .reset_index(name="Borrow_Count")
)

# Compare with total available titles
genre_availability = (
    books_df.groupby("Genre")["Book_ID"]
    .count()
    .reset_index(name="Total_Titles")
)

# Merge both
genre_analysis = genre_availability.merge(
    genre_borrow_counts, on="Genre", how="left"
)

genre_analysis["Borrow_Count"] = genre_analysis["Borrow_Count"].fillna(0)

# Calculate Borrow-to-Availability Ratio
genre_analysis["Borrow_Ratio"] = (
    genre_analysis["Borrow_Count"] / genre_analysis["Total_Titles"]
)

# Underrepresented = low borrow ratio
underrepresented_genres = genre_analysis.sort_values("Borrow_Ratio").head(10)

print(underrepresented_genres)


# When are peak borrowing periods?

transaction_df["Borrow_Date"] = pd.to_datetime(transaction_df["Borrow_Date"])

# Monthly trend
monthly_trend = (
    transaction_df.groupby(transaction_df["Borrow_Date"].dt.to_period("M"))
    .size()
    .reset_index(name="Borrows")
)

# Day of week trend
weekday_trend = (
    transaction_df.groupby(transaction_df["Borrow_Date"].dt.day_name())
    .size()
    .reset_index(name="Borrows")
)

print("Monthly borrowing trend:")
print(monthly_trend)

print("\nBorrowing by day of week:")
print(weekday_trend)


# How can we improve member engagement?

resource_dashboard = {
    "High_Demand_Books": high_demand_books,
    "Underrepresented_Genres": underrepresented_genres,
    "Monthly_Trend": monthly_trend,
    "Weekday_Trend": weekday_trend,
    "Member_Engagement": member_activity
}

print(resource_dashboard)




    Book_ID  Times_Borrowed                    Title       Genre
11     B012              48         The Desert Story   Biography
190    B192              45      Beauty: A Biography   Biography
269    B272              41      Desert: A Biography   Biography
143    B144              40              Tech Beauty  Technology
433    B439              39        Portrait of Power   Biography
428    B434              39      Philosophy of Honor  Philosophy
78     B079              39        The Lost Mountain     Fiction
7      B008              39      Gardens of the Mind  Philosophy
193    B195              39            The War Story   Biography
258    B261              39            Life of Peace   Biography
383    B388              38         The City Mystery     Fiction
4      B005              38        AI and the Future  Technology
86     B087              38        Fire: A Biography   Biography
173    B175              37       Rise of Innovation     History
6      B007              